In [ ]:
# 1. Librerías necesarias
import pandas as pd
import sqlite3
from IPython.display import display, Markdown

# 2. Cargar dataset
from google.colab import files
files.upload()

df = pd.read_csv("UCI_Credit_Card.csv")

# Renombramos la columna default.payment.next.month por default_payment
df.rename(columns={
    'default.payment.next.month': 'default_payment'
}, inplace=True)

# 3. Crear conexión a SQLite en memoria
conn = sqlite3.connect(":memory:")

# 4. Guardar el DataFrame como tabla SQL
df.to_sql("clientes", conn, index=False, if_exists="replace")

# 5. Función auxiliar para ejecutar y mostrar queries con título
def run_query(query, title):
    display(Markdown(f"### 🔹 {title}"))
    result = pd.read_sql(query, conn)
    display(result.head(10))  # mostrar primeras filas
    display(Markdown("---"))  # línea divisoria

# ============================
# 6. Queries con títulos
# ============================

# 1️ Total de clientes y porcentaje de impago
q1 = """
SELECT
    COUNT(*) AS total_clientes,
    SUM(default_payment) AS clientes_en_impago,
    ROUND(100.0 * SUM(default_payment) / COUNT(*), 2) AS porcentaje_impago
FROM clientes;
"""
run_query(q1, "1️ Total de clientes y porcentaje de impago")

# 2️ Edad promedio y límite de crédito según impago
q2 = """
SELECT
    default_payment,
    ROUND(AVG(AGE), 2) AS edad_promedio,
    ROUND(AVG(LIMIT_BAL), 2) AS limite_credito_promedio
FROM clientes
GROUP BY default_payment;
"""
run_query(q2, "2️ Edad promedio y límite de crédito por condición de impago")

# 3️ Distribución por sexo y condición de impago
q3 = """
SELECT
    SEX,
    default_payment,
    COUNT(*) AS num_clientes
FROM clientes
GROUP BY SEX, default_payment
ORDER BY SEX, default_payment;
"""
run_query(q3, "3️ Distribución de clientes por sexo y condición de impago")

# 4️ Impago según estado civil
q4 = """
SELECT
    MARRIAGE,
    COUNT(*) AS total_clientes,
    SUM(default_payment) AS casos_impago,
    ROUND(100.0 * SUM(default_payment) / COUNT(*), 2) AS tasa_impago_pct
FROM clientes
GROUP BY MARRIAGE
ORDER BY tasa_impago_pct DESC;
"""
run_query(q4, "4️ Tasa de impago por estado civil")

# 5️ Clientes de alto riesgo
q5 = """
SELECT
    ID,
    AGE,
    LIMIT_BAL,
    PAY_0,
    BILL_AMT1,
    PAY_AMT1
FROM clientes
WHERE AGE < 30
  AND PAY_0 >= 2
  AND default_payment = 1;
"""
run_query(q5, "5️ Clientes jóvenes en alto riesgo (retraso ≥2 meses y en impago)")

Saving UCI_Credit_Card.csv to UCI_Credit_Card (2).csv


### 🔹 1️ Total de clientes y porcentaje de impago

,total_clientes,clientes_en_impago,porcentaje_impago
0,30000,6636,22.12


---

### 🔹 2️ Edad promedio y límite de crédito por condición de impago

...

,MARRIAGE,total_clientes,casos_impago,tasa_impago_pct
0,3,323,84,26.01
1,1,13659,3206,23.47
2,2,15964,3341,20.93
3,0,54,5,9.26


---

### 🔹 5️ Clientes jóvenes en alto riesgo (retraso ≥2 meses y en impago)

,ID,AGE,LIMIT_BAL,PAY_0,BILL_AMT1,PAY_AMT1
0,1,24,20000.0,2,3913.0,0.0
1,23,26,70000.0,2,41087.0,2007.0
2,63,29,50000.0,2,24987.0,0.0
3,72,29,320000.0,2,58267.0,2500.0
4,118,26,80000.0,2,14029.0,2000.0
5,192,27,60000.0,2,19625.0,1342.0
6,229,23,50000.0,2,49758.0,2401.0
7,270,27,50000.0,3,13694.0,1700.0
8,289,25,100000.0,2,70896.0,3472.0
9,361,23,500000.0,4,507726.0,10000.0


---